Virginia updates at noon EDT daily.

In [1]:
from selenium import webdriver
import time
import pandas as pd
import pendulum
import re
import cv2
import time
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox) # linux only
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")
chrome_options.add_argument("user-agent=[Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:73.0) Gecko/20100101 Firefox/73.0]")

In [2]:
from PIL import Image
from pytesseract import image_to_string, image_to_data, image_to_pdf_or_hocr
import base64

scrape_timestamp = pendulum.now().strftime('%Y%m%d%H%M%S')

In [3]:
def save_tb_image(canvas, filename):
    """get the canvas as a PNG base64 string"""
    canvas_base64 = driver.execute_script("return arguments[0].toDataURL('image/png').substring(21);", canvas)

    # decode
    canvas_png = base64.b64decode(canvas_base64)

    # save to a file
    with open(f"{filename}_{scrape_timestamp}.png", 'wb') as f:
        f.write(canvas_png)

In [4]:
snip = """/* global arguments */
(function (element, deltaY, offsetX, offsetY) {
    var box = element.getBoundingClientRect();
    var clientX = box.left + (offsetX || box.width / 2);
    var clientY = box.top + (offsetY || box.height / 2);
    var target = element.ownerDocument.elementFromPoint(clientX, clientY);

    for (var e = target; e; e = e.parentElement) {
        if (e === element) {
            target.dispatchEvent(new MouseEvent("mouseover", {
                view: window,
                bubbles: true,
                cancelable: true,
                clientX: clientX,
                clientY: clientY
            }));
            target.dispatchEvent(new MouseEvent("mousemove", {
                view: window,
                bubbles: true,
                cancelable: true,
                clientX: clientX,
                clientY: clientY
            }));
            target.dispatchEvent(new WheelEvent("wheel", {
                view: window,
                bubbles: true,
                cancelable: true,
                clientX: clientX,
                clientY: clientY,
                deltaY: deltaY
            }));
            return "";
        }
    }
    return "Element is not interactable";
}).apply(null, arguments);
"""

In [5]:
def simulate_wheel(element, deltaY=120, offsetX=0, offsetY=0):
    """Helper to simulate a mouse wheel scroll."""
    error = element._parent.execute_script(snip, element, deltaY, offsetX, offsetY)
    if error:
        raise WebDriverException(error)

In [6]:
driver = webdriver.Chrome('../20190611 - Parts recommendation/chromedriver', options=chrome_options)

In [7]:
# url = 'http://www.vdh.virginia.gov/coronavirus/'
# url = 'https://public.tableau.com/views/VirginiaCOVID-19Dashboard/VirginiaCOVID-19Dashboard?%3Aembed=y&%3AshowVizHome=no&%3Adisplay_count=y&%3Adisplay_static_image=y&%3AbootstrapWhenNotified=true'
url = 'https://public.tableau.com/views/VirginiaCOVID-19Dashboard/VirginiaCOVID-19Dashboard?%3Aembed=y&%3AshowVizHome=no&%3Adisplay_count=y&%3Adisplay_static_image=y&%3Adevice=desktop'

In [8]:
driver.get(url)

In [9]:
# Virginia uses Tableau iframe for data
tbl = driver.find_element_by_id("main-content")
time.sleep(5)

In [10]:
# grab the table with county level cases
datatbl = tbl.find_element_by_id('tabZoneId115')

In [11]:
# scroll data table into view
driver.execute_script("arguments[0].scrollIntoView(true);", datatbl)

Virginia uses a public Tableau dashboard to report total cases. Need to grab the data table's canvas element and OCR.

In [12]:
# control how many mouse wheel scrolls to perform.
scroll_cnt = 4

In [13]:
for i in range(scroll_cnt):
    # grab all canvas elements
    canvases = datatbl.find_elements_by_css_selector('canvas')

    # save image
    for idx, canvas in enumerate(canvases):
        save_tb_image(canvas, f'source_page_backup/VA_county_{i}_{idx}')

    # scroll canvas down
    simulate_wheel(canvases[1], 500)
    time.sleep(3)

Join images together so data table OCR works properly

In [14]:
for i in range(scroll_cnt):
    im1 = cv2.imread(f'source_page_backup/VA_county_{i}_1_{scrape_timestamp}.png')
    im2 = cv2.imread(f'source_page_backup/VA_county_{i}_2_{scrape_timestamp}.png')
    im_h = cv2.hconcat([im1, im2[:,500:700,:]]) # trimming down the width of the positive cases so teseract works better
    cv2.imwrite(f'source_page_backup/VA_county_{i}_{scrape_timestamp}_final.png', im_h)

OCR combined image. 

In [36]:
data = []
for i in range(scroll_cnt):
    tmp = image_to_string(f'source_page_backup/VA_county_{i}_{scrape_timestamp}_final.png', config='--psm 6').split('\n')
    data += [re.search('(.*) (\d*)$', row).groups() for row in tmp]

In [38]:
df = pd.DataFrame(set(data), columns=['county', 'positive_cases'])

In [41]:
df.to_csv('data/VA_county_{}.txt'.format(scrape_timestamp), sep='|', index=False)